In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [2]:
import h5py
from tenpy.tools import hdf5_io\

import os

In [3]:
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS

In [4]:
from tenpy.networks.site import SpinSite, SpinHalfSite
from tenpy.models.lattice import Chain
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel

In [5]:
class TransverseCluster(CouplingModel):
    def __init__(self, model_params):
        # Read out/set default parameters
        name = "Cluster Ising model"
        L = model_params.get('L', 2)
        B = model_params.get('B', 0)
        bc_MPS = model_params.get('bc_MPS', 'infinite')
        
        # sites
        site = SpinHalfSite(conserve=None)

        # lattice
        bc = 'periodic'
        lat = Chain(L, site, bc=bc, bc_MPS=bc_MPS)

        # initialize CouplingModel
        CouplingModel.__init__(self, lat)

        # add terms of the Hamiltonian
        self.add_onsite(-B, 0, 'Sigmax')
        self.add_multi_coupling(
            -1,
            [
                ('Sigmaz', -1, 0),
                ('Sigmax', 0, 0),
                ('Sigmaz', 1, 0),
            ]
        )

        # initialize H_MPO
        MPOModel.__init__(self, lat, self.calc_H_MPO())

In [6]:
model_infinite = TransverseCluster({})

In [7]:
psi0_inf = MPS.from_desired_bond_dimension(model_infinite.lat.mps_sites(), 1, bc=model_infinite.lat.bc_MPS)

In [8]:
psi0_inf.canonical_form()

In [9]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [10]:
eng0_inf = dmrg.TwoSiteDMRGEngine(psi0_inf, model_infinite, dmrg_params)
E0_inf, _ = eng0_inf.run()
resume_psi0_inf = eng0_inf.get_resume_data(sequential_simulations=True)

In [11]:
resume_psi0_inf

{'psi': <tenpy.networks.mps.MPS at 0x74e07328c750>,
 'sequential_simulations': True,
 'init_env_data': {'init_LP': <npc.Array shape=(2, 4, 2) labels=['vR*', 'wR', 'vR']>,
  'age_LP': 444,
  'init_RP': <npc.Array shape=(2, 4, 2) labels=['vL', 'wL', 'vL*']>,
  'age_RP': 444}}

In [12]:
psi0_inf.entanglement_entropy()

array([0.69314718, 0.69314718])

In [14]:
print(psi0_inf.get_B(0))

<npc.Array shape=(2, 2, 2) labels=['vL', 'p', 'vR']
charge=ChargeInfo([], [])
 +1 | +1 | -1 
0 []|0 []|0 []
2   |2   |2   
[[[ 0.73607057 -0.08206514]
  [ 0.07445073  0.6677743 ]]

 [[ 0.6677743  -0.07445073]
  [-0.08206514 -0.73607057]]]
>


In [15]:
print(psi0_inf.get_B(1))

<npc.Array shape=(2, 2, 2) labels=['vL', 'p', 'vR']
charge=ChargeInfo([], [])
 +1 | +1 | -1 
0 []|0 []|0 []
2   |2   |2   
[[[-0.78018037  0.03795534]
  [-0.03034093 -0.62366451]]

 [[-0.62366451  0.03034093]
  [ 0.03795534  0.78018037]]]
>


In [16]:
E0_inf

-1.0000000000002132

In [17]:
enlarge = 5

In [18]:
model_seg = model_infinite.extract_segment(enlarge=enlarge)
first, last = model_seg.lat.segment_first_last

In [19]:
psi0_seg = psi0_inf.extract_segment(first, last)

In [20]:
init_env_data = eng0_inf.env.get_initialization_data(first, last)

In [21]:
init_env_data

{'init_LP': <npc.Array shape=(2, 4, 2) labels=['vR*', 'wR', 'vR']>,
 'age_LP': 444,
 'init_RP': <npc.Array shape=(2, 4, 2) labels=['vL', 'wL', 'vL*']>,
 'age_RP': 444}

In [24]:
psi1_seg = psi0_seg.copy()

In [22]:
resume_psi1_seg = {'init_env_data': init_env_data}

In [26]:
type(init_env_data)

dict

In [27]:
psi1_seg.finite

True

In [28]:
model_seg.finite

AttributeError: 'TransverseCluster' object has no attribute 'finite'

In [29]:
model_seg.H_MPO.finite

False

In [ ]:
dmrg.TwoSiteDMRGEngine(psi1_seg, model_seg, dmrg_params, resume_data=resume_psi1_seg)
DMRGEngine.super().__init__(psi1_seg, model_seg, dmrg_params, resume_data=resume_psi1_seg)
Sweep.init_env(model_seg, resume_data=resume_psi1_seg, orthogonal_to=None)
Sweep._init_mpo_env(model_seg.H_MPO, init_env_data)
Sweep.MPOEnvironment(psi1_seg, H, psi1_seg, cache=cache, **init_env_data)
MPOEnvironment.super().__init__(psi1_seg, psi1_seg, cache, **int_env_data)

In [25]:
eng = dmrg.TwoSiteDMRGEngine(psi1_seg, model_seg, dmrg_params, resume_data=resume_psi1_seg)

AssertionError: 

In [ ]:
E1_s, _ = eng.run()